<a href="https://colab.research.google.com/github/andrewzhai/ASOS_project_haoyang/blob/master/T5%20and%20Gpt3%20genator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# set up git and install packages

In [1]:
!git clone https://github.com/andrewzhai/ASOS_project_haoyang.git

Cloning into 'ASOS_project_haoyang'...
remote: Enumerating objects: 3898, done.
remote: Counting objects: 100% (57/57), done.
remote: Compressing objects: 100% (15/15), done.
remote: Total 3898 (delta 44), reused 51 (delta 42), pack-reused 3841
Receiving objects: 100% (3898/3898), 252.36 MiB | 31.09 MiB/s, done.
Resolving deltas: 100% (1797/1797), done.
Checking out files: 100% (3353/3353), done.


In [2]:
!pip install transformers==2.8.0
!pip install --upgrade PyYAML==5.1.2  # old version couldn't sort the order of dictionary

     |████████████████████████████████| 563 kB 8.0 MB/s 
     |████████████████████████████████| 895 kB 44.8 MB/s 
     |████████████████████████████████| 5.6 MB 43.1 MB/s 
     |████████████████████████████████| 131 kB 72.5 MB/s 
     |████████████████████████████████| 1.2 MB 60.2 MB/s 
     |████████████████████████████████| 7.8 MB 56.7 MB/s 
     |████████████████████████████████| 79 kB 9.9 MB/s 
     |████████████████████████████████| 138 kB 75.4 MB/s 
     |████████████████████████████████| 127 kB 73.8 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
     |████████████████████████████████| 265 kB 8.2 MB/s

In [3]:
!pip show transformers

Name: transformers
Version: 2.8.0
Summary: State-of-the-art Natural Language Processing for TensorFlow 2.0 and PyTorch
Home-page: https://github.com/huggingface/transformers
Author: Thomas Wolf, Lysandre Debut, Victor Sanh, Julien Chaumond, Sam Shleifer, Google AI Language Team Authors, Open AI team Authors, Facebook AI Authors, Carnegie Mellon University Authors
Author-email: thomas@huggingface.co
License: Apache
Location: /usr/local/lib/python3.7/dist-packages
Requires: regex, sacremoses, boto3, requests, filelock, numpy, tokenizers, tqdm, sentencepiece
Required-by: 


In [4]:
import torch
print(torch.__version__)

1.9.0+cu102


In [7]:
'k-fold'

import numpy as np
from sklearn.model_selection import KFold
X = np.array(['a','b','c','d','e'])
y = np.array([1,2,3,4,5])
kf = KFold(n_splits=5)
kf.get_n_splits(X)
print(kf)
KFold(n_splits=2, random_state=None, shuffle=False)
for train_index, test_index in kf.split(X):
  print("TRAIN:", train_index, "TEST:", test_index)
  X_train, X_test = X[train_index], X[test_index]
  


KFold(n_splits=5, random_state=None, shuffle=False)
TRAIN: [1 2 3 4] TEST: [0]
TRAIN: [0 2 3 4] TEST: [1]
TRAIN: [0 1 3 4] TEST: [2]
TRAIN: [0 1 2 4] TEST: [3]
TRAIN: [0 1 2 3] TEST: [4]


# Data preprocessing for NLU original

1.   read data from yaml files
2.   clean data 
3.   5-fold cross validation





In [29]:
# read gpt3 data
import yaml
with open('/content/ASOS_project_haoyang/input_data/T5_nlu.yml','r') as f:
  data=yaml.safe_load(f)


f=[]
for i in data['nlu']:
  text=[]
  for j in i['examples'].split('\n'):     # get the clean data from yaml
    input=j.replace('- ','')
    text.append(input)
  f.append(text[:-1])


In [ ]:
for i in f:
  print(i)

5-fold CV

In [33]:
f_train=[[],[],[],[],[]]
f_test=[[],[],[],[],[]]
# print(f_train)
kf = KFold(n_splits=5)

for i in range(len(f)):
 
  X=np.array(f[i])
  kf.get_n_splits(X)

  
  KFold(n_splits=5, random_state=1, shuffle=True)
  num=0
  for train_index, test_index in kf.split(X):
    s1=[]
    s2=[]
    
    s1.append(X[train_index].tolist())
    # print('--------')
    # print(s1)
    s2.append(X[test_index].tolist())
    f_train[num].append(s1)
    f_test[num].append(s2)
    num=num+1
  
  

  # train,test=train_test_split(section, test_size=0.2, random_state=42)
  # f_train.append(train)  #80%
  # f_test.append(test)  #20%


In [35]:
for element in f_train:
  for i in element:
    print(i[0])
   

['What kind of fur do you use?', 'Does your company use real fur?', 'Is your fur cruelty free?', 'Are your products made of real animal fur?', 'Does your product contain real fur?', 'Is the fur on your product real?', 'Is your fur fake?', 'Is your fur real?', 'Does your company use real fur?', 'Is your fur cruelty free?']
['I need my parcel to arrive on a specific day (UK)', 'I need my parcel to arrive on a specific day (International)', 'I need my parcel to arrive on a specific date', 'I need my parcel to arrive on a specific time', 'I need my parcel to arrive on a specific time on a specific day', 'I will be out of the country on delivery date, can you deliver on another date?', 'Can you deliver my parcel on a specific date?', 'Can you deliver my parcel on a specific date and time?']
['I have changed my address, can you update it?', 'I was sent the wrong item how can I have it delivered to another address?', 'How do I have my order delivered to a different address?', 'I want to chang

## write nlu_original files

after the generation, in order to run in rasa, you will still need slightly modified, based on the NLU demo file in RASA blog

In [37]:
!mkdir gpt_nlu_original

In [38]:
count=0
for element in f_train:
  count=count+1
  for (i,j) in zip(element,data['nlu']):

    
    
    
    j['examples']=i[0]

  
  with open('/content/gpt_nlu_original/{}cv_original_train.yml'.format(count), 'w') as outfile:
      yaml.dump(data, outfile, default_flow_style=False, sort_keys=False)

In [ ]:
count=0
for element in f_test:
  count=count+1
  for (i,j) in zip(element,data['nlu']):

    
    
    
    j['examples']=i[0]

  
  with open('/content/gpt_nlu_original/{}cv_original_test.yml'.format(count), 'w') as outfile:
      yaml.dump(data, outfile, default_flow_style=False, sort_keys=False)

In [ ]:
!zip -r /content/gpt_nlu_original.zip /content/gpt_nlu_original

  adding: content/gpt_nlu_original/ (stored 0%)
  adding: content/gpt_nlu_original/1cv_original_train.yml (deflated 80%)
  adding: content/gpt_nlu_original/5cv_original_train.yml (deflated 80%)
  adding: content/gpt_nlu_original/1cv_original_test.yml (deflated 75%)
  adding: content/gpt_nlu_original/2cv_original_test.yml (deflated 75%)
  adding: content/gpt_nlu_original/3cv_original_train.yml (deflated 80%)
  adding: content/gpt_nlu_original/4cv_original_train.yml (deflated 80%)
  adding: content/gpt_nlu_original/5cv_original_test.yml (deflated 75%)
  adding: content/gpt_nlu_original/2cv_original_train.yml (deflated 80%)
  adding: content/gpt_nlu_original/3cv_original_test.yml (deflated 76%)
  adding: content/gpt_nlu_original/4cv_original_test.yml (deflated 75%)


In [ ]:
# count=0
# for element in f_test:
#   count=count+1
#   for (i,j) in zip(element,c['nlu']):
    
#     l=[]
#     for k in i[0]:
#       print(k)
#       l.append(k.replace('- ',''))
      
#     j['examples']=l

#   print(c['nlu'])
#   with open('/content/{}cv_original_test.yml'.format(count), 'w') as outfile:
#       yaml.dump(c, outfile, default_flow_style=False, sort_keys=False)

# work on the augumented data

In [42]:
# sometimes this cell might throw an error, rerun the cell could solve the issues

import torch
from transformers import T5ForConditionalGeneration,T5Tokenizer


def set_seed(seed):
  torch.manual_seed(seed)
  if torch.cuda.is_available():
    torch.cuda.manual_seed_all(seed)

set_seed(42)


model = T5ForConditionalGeneration.from_pretrained('ramsrigouthamg/t5_paraphraser')
tokenizer = T5Tokenizer.from_pretrained('ramsrigouthamg/t5_paraphraser')

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print ("device ",device)
model = model.to(device)

sentence = "it is not the item I expected"
# sentence = "What are the ingredients required to bake a perfect cake?"
# sentence = "What is the best possible approach to learn aeronautical engineering?"
# sentence = "Do apples taste better than oranges in general?"


text =  "paraphrase: " + sentence + " </s>"


max_len = 256

encoding = tokenizer.encode_plus(text,pad_to_max_length=True, return_tensors="pt")
input_ids, attention_masks = encoding["input_ids"].to(device), encoding["attention_mask"].to(device)


# set top_k = 50 and set top_p = 0.95 and num_return_sequences = 3
beam_outputs = model.generate(
    input_ids=input_ids, attention_mask=attention_masks,
    do_sample=True,
    max_length=256,
    top_k=120,
    top_p=0.98,
    early_stopping=True,
    num_return_sequences=20
)


print ("\nOriginal Question ::")
print (sentence)
print ("\n")
print ("Paraphrased Questions :: ")
final_outputs =[]
for beam_output in beam_outputs:
    sent = tokenizer.decode(beam_output, skip_special_tokens=True,clean_up_tokenization_spaces=True)
    if sent.lower() != sentence.lower() and sent not in final_outputs:
        final_outputs.append(sent)

for i, final_output in enumerate(final_outputs):
    print("{}: {}".format(i, final_output))

device  cuda


Sequence can't be padded as no maximum length is specified and the model maximum length is too high.



Original Question ::
it is not the item I expected


Paraphrased Questions :: 
0: Is it the same item that I expected in the first place?
1: Is this the case I expected a bit so was this item/totem not worth it?
2: Is this the item I expected or expected?
3: What I think is most wrong with this product it is not the item I expected it to be. the content is overdone. you should think about how you are describing it to potential customers.
4: I haven't received the item I was expecting, so it's not the item I expected to get.
5: Is this item the thing I will receive?
6: It is not the item I expected. the item was not the item I expected. The manufacturer made a version with a different name, it's the same unit and it is the same. It is something I should try.
7: It is not the item I expected, it doesnt. (And was supposed to be with the pictures)
8: It is not the item I expected it to be. It is less than the cost to package it.
9: This is not the item I expected. :) I have nothing to lik

In [43]:
def generate_sentences(sentence,num,k,p):  #num: number of augumented sentence


  text =  "paraphrase: " + sentence + " </s>"
  print(text)


  max_len = 256

  encoding = tokenizer.encode_plus(text,pad_to_max_length=True, return_tensors="pt")
  input_ids, attention_masks = encoding["input_ids"].to(device), encoding["attention_mask"].to(device)

             
  # set top_k = 50 and set top_p = 0.95 and num_return_sequences = 3
  beam_outputs = model.generate(
      input_ids=input_ids, attention_mask=attention_masks,
      do_sample=True,
      max_length=256,
      top_k=k,
      top_p=p,
      early_stopping=True,
      num_return_sequences=30
  )
  final_outputs=[]
  for beam_output in beam_outputs:
      sent = tokenizer.decode(beam_output, skip_special_tokens=True,clean_up_tokenization_spaces=True)
      if sent.lower() != sentence.lower() and sent not in final_outputs:
          final_outputs.append(sent)

  return final_outputs[0:num]

test some parameters for T5 generator

In [44]:
generate_sentences('when can i get my delivery?',10,150,0.87)

Sequence can't be padded as no maximum length is specified and the model maximum length is too high.


paraphrase: when can i get my delivery? </s>


['How soon do I get delivery?',
 'How can I get my delivery?',
 'How do I get my delivery on time?',
 'How can I get delivery?',
 'When will you deliver your delivery?',
 'When will I get delivery?',
 'How do I get delivery on time?',
 'How do I get my delivery?',
 'How soon do you get your delivery?',
 'How do I know when I can get my delivery?']

In [ ]:
generate_sentences('when can i get my delivery?',10,50,0.99)

Sequence can't be padded as no maximum length is specified and the model maximum length is too high.


paraphrase: when can i get my delivery? </s>


['How do I get my delivery?',
 'How soon can I order delivered?',
 'When can I get my delivery on schedule?',
 'When can I have delivery?',
 'When will be my delivery?',
 'When will a delivery be done?',
 'How can I get my delivery?',
 'How long can I have my delivery?',
 'When do you get the delivery?',
 'When can I expect my delivery?']

In [ ]:
generate_sentences('when can i get my delivery?',10,50,0.89)

Sequence can't be padded as no maximum length is specified and the model maximum length is too high.


paraphrase: when can i get my delivery? </s>


['When will I receive my delivery?',
 'How should I know when I get my delivery?',
 'When can you get delivery?',
 'How do I get delivery?',
 'When can I have delivery?',
 'How do I get my delivery?',
 'How long will it take for delivery?',
 'When will you get your delivery?',
 'When will I get my delivery?',
 'How can you get delivery?']

In [ ]:
generate_sentences('when can i get my delivery?',10,100,0.89)

Sequence can't be padded as no maximum length is specified and the model maximum length is too high.


paraphrase: when can i get my delivery? </s>


['When do I get delivery?',
 'When can I get delivery service?',
 'How can I get delivery?',
 'How can I get my delivery to the house?',
 'How soon can I expect delivery?',
 'When will you get your delivery?',
 'When will I get my delivery?',
 'How often do I get my delivery?',
 'How can I get delivery information?',
 'When can I expect my delivery?']

In [ ]:
generate_sentences('when can i get my delivery?',10,100,0.99)

Sequence can't be padded as no maximum length is specified and the model maximum length is too high.


paraphrase: when can i get my delivery? </s>


['How do I get my delivery?',
 'How can I get delivery?',
 'When can I start my delivery?',
 'Where can I get delivery?',
 'How many days will you wait for delivery if you are not using your delivery service?',
 'Where do I get delivery?',
 'How quickly do I get my delivery order?',
 'When can I get delivery?',
 'How can I get my delivery?',
 'When will I receive my delivery?']

In [ ]:
generate_sentences('when can i get my delivery?',10,150,0.99)

Sequence can't be padded as no maximum length is specified and the model maximum length is too high.


paraphrase: when can i get my delivery? </s>


['When do I have to have delivery?',
 'When will I get my delivery?',
 'How can I get delivery?',
 'How soon should I get delivery?',
 'How do I get delivery day?',
 'How can I have my delivery?',
 'When do we get delivery?',
 'How can I get delivery today?',
 'How can I get my delivery anywhere near me?',
 'When is delivery coming?']

In [49]:
!mkdir augmented_data_t5

In [ ]:
for qq in range(1,6):
  #input 5 different files
  import yaml
  with open('/content/ASOS_project_haoyang/output_result_by_haoyang/data/original_data_5fold/{}cv_original_train.yml'.format(qq),'r') as f:
    data=yaml.safe_load(f)


  f_train=[]
  for i in data['nlu']:
    text=[]
    for j in i['examples'].split('\n'):     # get the clean data from yaml
      input=j.replace('- ','')
      text.append(input)
    f_train.append(text[:-1])

################################# read file below    
  c=data
  store=np.ones(32).tolist()
  import numpy as np
    

  count=0
  for i in f_train:
  
    arr=[]
    for sentence in i:
        l=[]
        # T5 data generator
        sentences=generate_sentences(sentence,12,150,0.87)
        print('----------------')
        print(sentence)
        l.append(sentence)
        l.extend(sentences) 
        new=[]
        print(l)
        ## store sentence
        for q in range(1,11):
            
          new.append(l[0:q+1])
        arr.append(new)
        
        
    store[count]=arr
    count=count+1


      ############################################################################
  d=[]
  for i in store:
    print(np.array(i).shape )
    d.append(np.array(i).shape[0])   



  for num in range(0,10):
    c=data
      
    count=0
    for (i,j,n) in zip(f_train,c['nlu'],d):
      new=[]
      print('*************')
      print(j['intent'])
      for q in range(n):
        new.extend(store[count][q][num])
          
            
          
      
        
        
      count=count+1
      print('-----------')
      print(len(new))
      j['examples']=new
        
    print(c['nlu'])
      

    with open('augmented_data_t5/{}cv_{}augumented_fixed.yml'.format(qq,num+1), 'w') as outfile:
          yaml.safe_dump(c, outfile, default_flow_style=False, sort_keys=False)

Sequence can't be padded as no maximum length is specified and the model maximum length is too high.


paraphrase: What kind of fur do you use? </s>


Sequence can't be padded as no maximum length is specified and the model maximum length is too high.


----------------
What kind of fur do you use?
['What kind of fur do you use?', 'What kind of fur does one use?', 'What type of fur do you use?', 'What kind of fur do you like?', 'What kind of fur does a human use?', 'What kind of fur does one like?']
paraphrase: Does your company use real fur? </s>


Sequence can't be padded as no maximum length is specified and the model maximum length is too high.


...

----------------
My voucher code has been used but I havent bought anything
['My voucher code has been used but I havent bought anything', 'My voucher code has been used but I havent bought anything.', "My voucher code has been used but I havent bought anything. I can't use it at all in store.", "My voucher code has been used but I haven't bought anything.", 'I have used my voucher code but I havent bought anything and it still says voucher code is used.', "My voucher code has been used but I haven't bought anything. I have used my voucher code before but still not bought anything. Is that normal?", 'Is my voucher code used?', "My voucher code has been used but I haven't bought anything yet.", "My voucher code has been used but I haven't bought anything, what have I done?", "My voucher code has been used but I haven't bought anything. Is there anything I can buy with it?", "My voucher code has been used but I havent bought anything yet. If it is something that I have bought from my par

Sequence can't be padded as no maximum length is specified and the model maximum length is too high.


----------------
My voucher code hasn't been used, I have a discount code
["My voucher code hasn't been used, I have a discount code", "My voucher code hasn't been used. I have a discount code, is it a code I didn't know?", "My voucher code hasn't been used, I have a discount code. My voucher code hasn't been used. I haven't used it yet.", "My voucher code hasn't been used, I have a discount code. Is that what I was trying to do?", "My voucher code hasn't been used, I have a discount code.", "My voucher code hasn't been used, I have a discount code!", "My voucher code hasn't been used, I have a discount code. I want to try and use another voucher code.", "My voucher code hasn't been used, I have a discount code?", "My voucher code hasn't been used, I have a discount code. How do I use it?", "My voucher code hasn't been used, I have a discount code. I have no coupon code.", "My voucher code hasn't been used, I have a discount code. How can I get the code back?", "I forgot my voucher cod

Sequence can't be padded as no maximum length is specified and the model maximum length is too high.


----------------
My voucher code says it's already been used
["My voucher code says it's already been used", "My voucher code says it's already been used. But if I'm paying something on there, what are my options?", "My voucher code says it's already been used. Is it really correct?", "My voucher code says it's already been used.", "My voucher code says it's already been used.      ", "I don't see how to enter voucher code if it says it has already been used.", "My voucher code says it's already been used. But I can't be more correct!", "My voucher code says it's already been used and I don't have to enter the code to see it in my account. How can I get the voucher code back?", "My voucher code says it's already been used. I want to use it now.", "My voucher code says I've already been used!", 'Is it true that my voucher code has already been used?', 'My voucher code says it has already been used.', 'I found my voucher code has been used but I forgot to enter it.']
paraphrase: My vouch

# Gpt3 data augmentation

Add gpt3 augmented data into 5-fold original files

In [ ]:
import numpy as np

In [ ]:
!mkdir augmented_data_gpt

In [ ]:
# read and clean gpt data
import yaml
with open('/content/ASOS_project_haoyang/input_data/gpt3_augmented.yml','r') as f:
  data_gpt=yaml.safe_load(f)


gpt=[]
for i in data_gpt['nlu']:
  text=[]
  for j in i['examples'].split('\n'):     # get the clean data from yaml
    input=j.replace('- ','')
    text.append(input)
  gpt.append(text[:-1])

In [ ]:
for qq in range(1,6):

  import yaml
  with open('/content/ASOS_project_haoyang/output_result_by_haoyang/data/original_data_5fold/{}cv_original_train.yml'.format(qq),'r') as f:
    data=yaml.safe_load(f)


  f_train=[]
  for i in data['nlu']:
    text=[]
    for j in i['examples'].split('\n'):     # get the clean data from yaml
      input=j.replace('- ','')
      text.append(input)
    f_train.append(text[:-1])

################################# read file below    
  c=data
  store=np.ones(32).tolist()
  import numpy as np
    

  count=0
  for (i,j) in zip(f_train,gpt):
  
    arr=[]
    # for sentence in i:
    #     l=[]
    #     sentences=
    #     print('----------------')
    #     print(sentence)
    #     l.append(sentence)
    #     l.extend(sentences) 
    #     new=[]
    #     print(l)
    #     ## store sentence
    pool=np.array(j)
    np.random.shuffle(pool)
    j=pool.tolist()
    new=[]
    for q in range(1,11):

 
      new.append(j[0:10*q])
    arr.append(new)
        
        
    store[count]=arr
    count=count+1


      ############################################################################
  d=[]
  for i in store:
    print(np.array(i).shape )
    d.append(np.array(i).shape[0])   



  for num in range(0,10):
    c=data
      
    count=0
    for (i,j,n) in zip(f_train,c['nlu'],d):
      new=[]
      print('*************')
      print(j['intent'])

      for q in range(n):

        new.extend(store[count][q][num])
          
            
          
      
        
        
      count=count+1
      shuffle=np.array(new)
      np.random.shuffle(shuffle)
      new=shuffle.tolist()
      j['examples']=new
        
    print(c['nlu'])
      

    with open('/content/augmented_data_gpt/{}cv_{}augumented_fixed.yml'.format(qq,num+1), 'w') as outfile:
          yaml.safe_dump(c, outfile, default_flow_style=False, sort_keys=False)
     

In [ ]:
!zip -r /content/gpt_nlu_original.zip /content/gpt_nlu_original

  adding: content/gpt_nlu_original/ (stored 0%)
  adding: content/gpt_nlu_original/1cv_original_train.yml (deflated 80%)
  adding: content/gpt_nlu_original/5cv_original_train.yml (deflated 80%)
  adding: content/gpt_nlu_original/1cv_original_test.yml (deflated 75%)
  adding: content/gpt_nlu_original/2cv_original_test.yml (deflated 75%)
  adding: content/gpt_nlu_original/3cv_original_train.yml (deflated 80%)
  adding: content/gpt_nlu_original/4cv_original_train.yml (deflated 80%)
  adding: content/gpt_nlu_original/5cv_original_test.yml (deflated 75%)
  adding: content/gpt_nlu_original/2cv_original_train.yml (deflated 80%)
  adding: content/gpt_nlu_original/3cv_original_test.yml (deflated 76%)
  adding: content/gpt_nlu_original/4cv_original_test.yml (deflated 75%)
